In [1]:
import numpy as np
import pandas as pd
import numpy
import matplotlib.pyplot as plt

#Fix colormath depreciated numpy method
def patch_asscalar(a):
    return a.item()
setattr(numpy, "asscalar", patch_asscalar)

#Import data
all_colors = pd.read_csv("data/all-colors.csv")
chosen_colors = pd.read_csv("data/chosen-colors.csv")

#Remove chosen colors
condition = all_colors["Shirt Color Name"].isin(chosen_colors["Shirt Color"])
remaining_colors = all_colors.drop(all_colors[condition].index, inplace=False)

#Confirm removal
# print(len(remaining_colors))
# print(len(all_colors))
# print(len(chosen_colors))

# Find Most Distant Colors to Chosen Colors

In [2]:
import colormath.color_objects as co
import colormath.color_diff as cdiff

def compute_average_distance(row):
    color_Lab = None

    color_Lab = co.LabColor(*map(float, row["L*a*b Value"].split(", ")))

    distances = []

    #Compute all distances from chosen colors
    for index, row_remaining in chosen_colors.iterrows():
        other_color = co.LabColor(*map(float, row_remaining["L*a*b Value"].split(", ")))

        #Newer, asymmetric formula, weighted for fabrics, using
        #distance = cdiff.delta_e_cmc(other_color, color_Lab , pl=2, pc=1)

        #Older symmetric formula, weighted for fabrics
        distance = cdiff.delta_e_cie2000(color_Lab, other_color,Kl=2, Kc=1, Kh=1)

        distances += [distance]

    average_distance = sum(distances)/len(remaining_colors)

    row["Distance"] = average_distance
    return row

remaining_colors = remaining_colors.apply(compute_average_distance, axis=1)

In [3]:
sorted_remaining = remaining_colors.sort_values(by = "Distance", ascending=False)
sorted_remaining.iloc[:].head(10)

,Shirt Color Name,PMS Color Code,L*a*b Value,Hex,Distance
47,Teal,3262C,"66.44, -59.33, -12.05",00BFB2,7.545582
73,Heather Kelly,340 C,"51.80, -65.36, 14.55",00965E,7.436395
50,Turquoise,638C,"63.99, -33.79, -34.95",00AFD7,7.312165
24,Maize Yellow,7548CP,"83.14, 5.01, 85.02",F2CA00,7.287751
104,Heather Yellow,100 CP,"92.09, -6.83, 62.42",F6EB69,7.094500
96,Heather Sea Green,2398 C,"66.81, -46.18, -10.83",00BAB3,6.989276
37,Poppy,2347C,"49.45, 75.12, 67.21",E10600,6.858418
20,Kelly,341C,"43.23, -48.36, 9.96",007A53,6.852545
83,Heather Orange,1645 C,"65.69, 58.38, 58.65",FF6A39,6.630476
34,Orange,1645C,"65.69, 58.38, 58.65",FF6A39,6.630476


# Attempting to Make 2D Graphs

In [4]:
#Orange to Gold

orange = all_colors[all_colors["Shirt Color Name"] == "Orange"]
gold = all_colors[all_colors["Shirt Color Name"] == "Gold"]
burnt_orange = all_colors[all_colors["Shirt Color Name"] == "Burnt Orange"]

comparison_list = [gold, orange, burnt_orange]

comparison_matrix = [[0]*3 for i in range(3)]

for i in range(len(comparison_list)):
    for j in range(len(comparison_matrix)):
        #I really should just fix the tables
        target_color = co.LabColor(*map(float,comparison_list[i].iloc[0]["L*a*b Value"].split(", ")))
        comparison_color = co.LabColor(*map(float,comparison_list[j].iloc[0]["L*a*b Value"].split(", ")))

        #Newer formula
        distance = cdiff.delta_e_cmc(target_color, comparison_color, pl=2, pc=1)

        comparison_matrix[i][j] = distance

print("    Gold    Orange    Burnt Orange")
for row in comparison_matrix:
    print(row)

    Gold    Orange    Burnt Orange
[0.0, 24.24013342020266, 17.480586695643115]
[26.402279861193914, 0.0, 9.887678740145626]
[22.286845806880518, 11.611165190695397, 0.0]


In [5]:
import networkx as nx

# Let's make an adjacency matrix representing color distances to/from each other
adjacency_matrix = pd.DataFrame(np.zeros(shape=(16, 16)),
columns = chosen_colors["Shirt Color"].unique(), index=chosen_colors["Shirt Color"].unique())

for index_a, row_a in chosen_colors.iterrows():
    for index_b, row_b in chosen_colors.iterrows():
        color_a = co.LabColor(*map(float, row_a["L*a*b Value"].split(", ")))
        color_b = co.LabColor(*map(float, row_b["L*a*b Value"].split(", ")))

        distance = cdiff.delta_e_cie2000(color_a, color_b, Kl=2, Kc=1, Kh=1)

        adjacency_matrix.at[row_a["Shirt Color"], row_b["Shirt Color"]] = distance


# Plot the Chosen Colors in 3D!

In [13]:
import plotly.graph_objects as go

# Read the CSV file
# Assuming your CSV has columns: "Shirt Color", "Pantone Color", "L*a*b Value", "HEX"
def generate_lab_3d(df, figure_title, output_filename):
    # Parse the L*a*b values from the string format
    df[['L', 'a', 'b']] = df['L*a*b Value'].str.split(', ', expand=True).astype(float)

    # Create the 3D scatter plot
    fig = go.Figure(data=[go.Scatter3d(
        x=df['a'],        # a* axis (green-red)
        y=df['L'],        # L* axis (lightness)
        z=df['b'],        # b* axis (blue-yellow)
        mode='markers+text',
        text=df['Shirt Color'],
        hovertemplate=
            '<b>%{text}</b><br>' +
            'L*: %{y:.1f}<br>' +
            'a*: %{x:.1f}<br>' +
            'b*: %{z:.1f}<br>',
        marker=dict(
            size=10,
            color=['#' + hex.strip() for hex in df['HEX']],  # Use actual hex colors
            opacity=0.9
        ),
        textposition='top center'
    )])

    # Update the layout
    fig.update_layout(
        template="plotly",
        title=figure_title,
        scene=dict(
            aspectratio=dict(x=1, y=1, z=1),
            xaxis_title='a* (green to red)',
            yaxis_title='L* (black to white)',
            zaxis_title='b* (blue to yellow)',
            # Set appropriate ranges for each axis
            xaxis=dict(range=[-128, 128]),
            yaxis=dict(range=[0, 100]),
            zaxis=dict(range=[-128, 128])
        ),
        width=700,
        height=700,
        showlegend=False
    )

    # Add better camera angle
    fig.update_layout(scene_camera=dict(
        up=dict(x=0, y=1, z=0),
        center=dict(x=0, y=0, z=0),
        eye=dict(x=1.5, y=1.5, z=1.5)
    ))

    # Show the plot
    # fig.show()

    # Optionally save to HTML file
    fig.write_html(output_filename + ".html")

In [14]:
df = pd.read_csv('data/chosen-colors.csv')

generate_lab_3d(df=df, figure_title="Chosen Colors", output_filename="Chosen_Colors_test")
#
#
# #Clean data
# sorted_fixed_columns = sorted_remaining.drop('Distance', axis=1)
# sorted_fixed_columns.columns = df.columns
#
# def prepend_star(row):
#     row["Shirt Color"] = "*" + row["Shirt Color"]
#     return row
#
# sorted_fixed_columns = sorted_fixed_columns.apply(prepend_star, axis=1)
#
# df = pd.concat([df, sorted_fixed_columns.iloc[0:10]], ignore_index=True)
#
# generate_lab_3d(df=df, figure_title="Additions from CIE", output_filename="CIE_Additions")